In [54]:
import pandas as pd
import numpy as np
import os

In [55]:
# directory = '../data/wisdm-dataset.zip'
# output_directory = '../data/'

# os.listdir(output_directory)

# import zipfile
# with zipfile.ZipFile(directory, 'r') as zip_ref:
#     zip_ref.extractall(output_directory)

In [56]:
import string

letters = ['A','B',
 'C', 'D',
 'E', 'F',
 'G', 'H',
 'I', 'J',
 'K', 'L',
 'M', 'O',
 'P', 'Q',
 'R', 'S']



labels = ['Walking','Jogging','Stairs','Sitting','Standing','Typing','Brushing Teeth','Eating Soup',
'Eating Chips','Eating Pasta','Drinking from Cup','Eating Sandwich','Kicking (Soccer Ball)','Playing Catch w/Tennis Ball','Dribblinlg (Basketball)','Writing',
'Clapping','Folding Clothes']
activity_dict_to_label = {a[0]:a[1] for a in list(zip(letters,labels))}

activity_dict_to_index = {a[0]:i for i,a in enumerate(letters)}

In [57]:
activity_dict_to_label

{'A': 'Walking',
 'B': 'Jogging',
 'C': 'Stairs',
 'D': 'Sitting',
 'E': 'Standing',
 'F': 'Typing',
 'G': 'Brushing Teeth',
 'H': 'Eating Soup',
 'I': 'Eating Chips',
 'J': 'Eating Pasta',
 'K': 'Drinking from Cup',
 'L': 'Eating Sandwich',
 'M': 'Kicking (Soccer Ball)',
 'O': 'Playing Catch w/Tennis Ball',
 'P': 'Dribblinlg (Basketball)',
 'Q': 'Writing',
 'R': 'Clapping',
 'S': 'Folding Clothes'}

In [58]:
activity_dict1 = {'No Label':-1,'Sitting':0,'Stairs':1,'Soccer':2,
                'Cycling':3,'Driving':4,'Lunch':-1,'Walking':5,
                'Working':-1}
activity_dict_to_index = {'A': 3, 'B': 2, 'C': 1, 'D': 0, 'E': 0, 'F': 0,
                          'G': -1, 'H': -1, 'I': -1, 'J': -1, 'K': -1, 'L': -1,
                          'M': 4, 'O': 4, 'P': 4, 'Q': 0, 'R': -1, 'S': -1}


activity_list = ['Stationery','Stairs','Exercise','Walking','Sports']
indexes =       [    0,        1,        2,        3,          4,         5]

In [59]:
from datetime import datetime
from scipy.stats import mode

def get_individual_windows(filelists,fs = 20,window_size = 20,factor=2):
    step = fs*window_size//factor
    y_activity = []
    y_participant = []
    X = []
    y_time = []
    y_hr =[]
    for i,f in enumerate(filelists):
        data = pd.read_csv(f,names=['user','activity','timestamp','x','y','z'],delimiter=',')
        print(data.activity.unique())
        data['activity_index'] = data['activity'].apply(lambda a:activity_dict_to_index[a])
        data['user'] = i
        data['timestamp'] = data['timestamp'].apply(lambda a:a/1e6)
        data['z'] = data['z'].apply(lambda a:np.float32(a[:-1]))
        data  = data.sort_values('timestamp').reset_index(drop=True)
        for k in range(0,data.shape[0]-fs*window_size,step):
            temp = data[k:k+fs*window_size]
            if mode(temp['activity_index'].values)[0][0]==-1:
                continue
            y_activity.append(mode(temp['activity_index'].values)[0][0])
            y_participant.append(i)
            X.append(temp[['x','y','z']].values.reshape(1,-1,3))
            y_time.append(np.mean(data['timestamp']))
            y_hr.append(60)
    X = np.concatenate(X)
    y_participant = np.array(y_participant)
    y_activity = np.array(y_activity)
    y_time = np.array(y_time)
    y_hr = np.array(y_hr)
    print(X.shape,y_hr.shape,y_participant.shape,y_activity.shape,y_time.shape)
    return X,y_hr,y_participant,y_activity,y_time

window_size = 20
filepath = '../data/wisdm-dataset/raw/watch/accel/'
filelists = [os.path.join(filepath,a) for a in os.listdir(filepath) if a[0]=='d']
a = get_individual_windows(filelists,window_size=window_size)
import pickle
pickle.dump(a,open('../data/wisdm_individual_windows_'+str(window_size)+'.p','wb'))

['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'O' 'P' 'Q' 'R' 'S']
['A' 'B' 'C' 'D' 'E' 'F' 'G' 'H' 'I' '

In [ ]:
65830/20/3600

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
# plt.hist(a[0][:,:,0])
plt.plot(a[0][0])
plt.show()
a[0].shape

In [ ]:
np.max(a[0])*2/66